<a href="https://colab.research.google.com/github/machine-perception-robotics-group/JDLALectureNotebooks/blob/master/notebooks/xx_neural_network_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ニューラルネットワークによる回帰

---

## 目的

多層パーセプトロンを用いて，糖尿病の進行度を予測する回帰を行う．


## モジュールのインポート

プログラムの実行に必要なモジュールをインポートします．

In [ ]:
from sklearn import datasets

import torch
import torch.nn as nn
import torch.optim as optim


## データセットの読み込み

実験に使用するデータセットを読み込みます．

今回は**Diabetes Dataset**を用いて回帰を行います．
Diabetes Datasetは，糖尿病患者の一年後の進行度を予測するデータセットです．

説明変数となる特徴量は，年齢，性別，BMI，平均血圧などの計10個の数値から構成されており，目的変数はベースラインとなる測定時から1年後の糖尿病の進行度を示す数値となっています．

今回は，scikit-learnで用意されているデータセットクラスからDiabetes Datasetを呼び出して使用します．



In [ ]:
# データセットの読み込み
diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)

# データ（配列）の確認
print(type(diabetes_X))
print(type(diabetes_y))
print(diabetes_X.shape)
print(diabetes_y.shape)

print(diabetes_X[0])
print(diabetes_y[0])

## データセットクラスの構築

上記のDiabetes Datasetを用いて，PyTorchのNeural Networkを学習できるよう，データセットクラスを構築します．

In [ ]:
class DiabetesDataset(torch.utils.data.Dataset):

    def __init__(self, train=True):
        self.n_train =350
        self.n_test = 92

        self.train =train

        self.X, self.y = datasets.load_diabetes(return_X_y=True)
        self.X = torch.tensor(self.X, dtype=torch.float32)
        self.y = torch.tensor(self.y, dtype=torch.float32)

        if self.train:
            self.X = self.X[:self.n_train]
            self.y = self.y[:self.n_train]
        else:
            self.X = self.X[self.n_train:]
            self.y = self.y[self.n_train:]

    def __getitem__(self, index):
        return self.X[index], self.y[index] / 346.0

    def __len__(self):
        return len(self.X)

## ネットワークモデルの定義

次に，ニューラルネットワークを定義します．
ここでは，PyTorchを使用して多層パーセプトロンを構築します．

In [ ]:
class MLP(nn.Module):
  def __init__(self, n_hidden):
    super().__init__()
    self.l1=nn.Linear(10, n_hidden)
    self.l2=nn.Linear(n_hidden, n_hidden)
    self.l3=nn.Linear(n_hidden,1)

  def forward(self, x):
    x = self.l1(x)
    x = torch.relu(x)
    x = self.l2(x)
    x = torch.relu(x)
    x = self.l3(x)
    return x

## 学習

上で定義したデータセットクラスとネットワークモデルを使用して学習します．

誤差関数にはMSE LossとMAE Loss（nn.L1Loss）を準備します．

In [ ]:
# prompt: 学習プログラムを書いてください

# ハイパーパラメータの設定
n_hidden = 128
epochs = 100
batch_size = 10

# データセットの読み込み
train_dataset = DiabetesDataset(train=True)
test_dataset = DiabetesDataset(train=False)

# データローダーの設定
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# モデルの定義
model = MLP(n_hidden)

# オプティマイザーの定義
optimizer = optim.Adam(model.parameters())

# 損失関数の定義
criterion = nn.MSELoss() # MSE Loss
# criterion = nn.L1Loss()  # MAE Loss

# 学習ループ
for epoch in range(1, epochs+1):
  # 訓練モード
  model.train()

  # ミニバッチ学習
  for x, y in train_dataloader:
    # 勾配の初期化
    optimizer.zero_grad()

    output = model(x)
    loss = criterion(output, y)

    # 勾配の計算
    loss.backward()
    optimizer.step()

  # 評価モード
  model.eval()

  # テストデータでの損失の計算
  with torch.no_grad():
    test_loss = 0
    for x, y in test_dataloader:
      output = model(x)
      test_loss += criterion(output, y).item()

  # 損失の出力
  if epoch % 10 == 0:
    print(f"Epoch: {epoch}, Loss: {loss.item():.4f}, Test Loss: {test_loss:.4f}")